In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


os.chdir("..")

In [2]:
data = pd.read_csv("dataset/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Data Preprocessing

## Drop unnecessary columns
data=data.drop(columns=["RowNumber", "CustomerId", "Surname"])

## Convert categorical variables to numerical
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import OneHotEncoder
onthot_encoder_geo = OneHotEncoder()
geo_encoded = onthot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(), columns=onthot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## Concatenate the new columns to the original dataframe
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Save encoders
os.makedirs("models", exist_ok=True)
with open("models/label_encoder_gender.pkl", "wb") as file:
  pickle.dump(label_encoder_gender, file)
with open("models/onehot_encoder_geo.pkl", "wb") as file:
  pickle.dump(onthot_encoder_geo, file)

In [9]:
## Divide the data into features and target variable
X = data.drop(columns=["Exited"])
y = data["Exited"]

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Save the scaler
with open("models/scaler.pkl", "wb") as file:
  pickle.dump(scaler, file)

In [10]:
# ANN Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [11]:
X_train.shape[1]

12

In [12]:
## Build the ANN model
model = Sequential([
  Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1
  Dropout(0.2),
  Dense(32, activation='relu'), ## HL2
  Dropout(0.2),
  Dense(1, activation='sigmoid'), ## Output layer
])

d:\Project\customer_churn\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
## Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [15]:
## Setup TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [16]:
## Setup EarlyStopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [17]:
## Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks=[tensorflow_callback, early_stopping_callback], verbose=1)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7678 - loss: 0.5103 - val_accuracy: 0.8255 - val_loss: 0.4012
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8181 - loss: 0.4272 - val_accuracy: 0.8455 - val_loss: 0.3734
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8392 - loss: 0.3963 - val_accuracy: 0.8510 - val_loss: 0.3559
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8421 - loss: 0.3862 - val_accuracy: 0.8595 - val_loss: 0.3470
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8488 - loss: 0.3659 - val_accuracy: 0.8605 - val_loss: 0.3428
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8496 - loss: 0.3673 - val_accuracy: 0.8625 - val_loss: 0.3413
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8522 - loss: 0.3634 - val_accuracy: 0.8605 - val_loss: 0.3418
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8552 - loss: 0.3603 - val_accu

In [18]:
## Save the model
model.save("models/model.keras")

In [21]:
os.getcwd()

'd:\\Project\\customer_churn'

In [23]:
## Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
%tensorboard --logdir logs/fit/20250519-214447


Reusing TensorBoard on port 6007 (pid 11428), started 0:00:46 ago. (Use '!kill 11428' to kill it.)

In [25]:
## Load the model
from tensorflow.keras.models import load_model
model = load_model("models/model.keras")
## Load the scaler
with open("models/scaler.pkl", "rb") as file:
  scaler = pickle.load(file)
## Load the encoders
with open("models\label_encoder_gender.pkl", "rb") as file:
  label_encoder_gender = pickle.load(file)
with open("models\onehot_encoder_geo.pkl", "rb") as file:
  onthot_encoder_geo = pickle.load(file)



<>:8: SyntaxWarning: invalid escape sequence '\l'
<>:10: SyntaxWarning: invalid escape sequence '\o'
<>:8: SyntaxWarning: invalid escape sequence '\l'
<>:10: SyntaxWarning: invalid escape sequence '\o'
C:\Users\soumasnigdha\AppData\Local\Temp\ipykernel_23528\58831972.py:8: SyntaxWarning: invalid escape sequence '\l'
  with open("models\label_encoder_gender.pkl", "rb") as file:
C:\Users\soumasnigdha\AppData\Local\Temp\ipykernel_23528\58831972.py:10: SyntaxWarning: invalid escape sequence '\o'
  with open("models\onehot_encoder_geo.pkl", "rb") as file:


In [42]:
## Example input data
input_data = {
  'CreditScore': 600,
  'Geography': 'France',
  'Gender': 'Male',
  'Age': 40,
  'Tenure': 3,
  'Balance': 60000,
  'NumOfProducts': 2,
  'HasCrCard': 1,
  'IsActiveMember': 1,
  'EstimatedSalary': 50000
}

In [43]:
## Encode the new input data
geo_encoded = onthot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onthot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

d:\Project\customer_churn\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
input_data = pd.concat([pd.DataFrame([input_data]), geo_encoded_df], axis=1)

In [45]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [46]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])

In [47]:
input_data.drop(columns=['Geography'], inplace=True, axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [48]:
## Scale the input data
input_data = scaler.transform(input_data)
input_data

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [49]:
## Make prediction
prediction = model.predict(input_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


array([[0.03837079]], dtype=float32)

In [50]:
prediction_proba = prediction[0][0]
prediction_proba

0.038370788

In [51]:
if prediction_proba > 0.5:
    print("The customer is likely to exit.")
else:
    print("The customer is likely to stay.")

The customer is likely to stay.
